# なぜ、メジャーリーグ貧乏球団から線形回帰を学ぶか？

* TensorFlow の練習になるから
* 基礎は重要だから
  * 最尤原理から線形回帰を捉え直すと、ロジスティック回帰とディープラーニングに自然につながる。


* オークランドアスレチックスの話が面白いから
  * ["Moneyball:The Power of Sport Analytics" edX MITx:15071x The Analytics Edge](https://courses.edx.org/courses/course-v1:MITx+15.071x_3+1T2016/courseware/f8d71d64418146f18a066d7f0379678c/6324edb8a22c4e35b937490647bfe203/?activate_block_id=block-v1%3AMITx%2B15.071x_3%2B1T2016%2Btype%40sequential%2Bblock%406324edb8a22c4e35b937490647bfe203)
    * 2000年代初頭、貧乏球団オークランドアスレチックスは統計的推論を駆使することで、プレーオフ常連となった。
       * 当時、「野球はスポーツではなく金銭ゲーム」と言われ、良い選手はことごとく金満球団へ引き抜かれていた。
       * そんななか、オークランドアステチックスが選手に支払っている総年棒は、金満球団レッドソックスの1/3だった。にもかかわらず、勝率はほぼ同じ！

In [1]:
import tensorflow as tf

from math import *
import numpy as np
import pandas as pd
import urllib
import StringIO
import matplotlib.pyplot as plt
%matplotlib inline

# 線形回帰

2002年にオークランドアスレチックスのビリー・ビーンが行った解析を、線形回帰で再現してみる。

* すなわち、2002年よりも前のデータを使って、統計的モデルを作ってみる。


## メジャーリーグの試合データ

[Baseball-Reference.com](http://www.baseball-reference.com/)が提供しているデータを、edXのサイトからダウンロード。

In [2]:
baseball = pd.read_csv("https://d37djvu3ytnwxt.cloudfront.net/assets/courseware/v1/dfb1bb5463c388fb167745888e3a6dd9/asset-v1:MITx+15.071x_3+1T2016+type@asset+block/baseball.csv")
baseball.head()

,Team,League,Year,RS,RA,W,OBP,SLG,BA,Playoffs,RankSeason,RankPlayoffs,G,OOBP,OSLG
0,ARI,NL,2012,734,688,81,0.328,0.418,0.259,0,NaN,NaN,162,0.317,0.415
1,ATL,NL,2012,700,600,94,0.320,0.389,0.247,1,4.0,5.0,162,0.306,0.378
2,BAL,AL,2012,712,705,93,0.311,0.417,0.247,1,5.0,4.0,162,0.315,0.403
3,BOS,AL,2012,734,806,69,0.315,0.415,0.260,0,NaN,NaN,162,0.331,0.428
4,CHC,NL,2012,613,759,61,0.302,0.378,0.240,0,NaN,NaN,162,0.335,0.424


2002年よりも前のデータを取り出しておく。

In [3]:
moneyball = baseball[ baseball.Year < 2002 ]

## 得点のモデル
$$y = b + {\bf x \cdot w}$$

* 従属変数
  * $y$ : シーズン中の総得点
* 独立変数 
  * ベクトル ${\bf x} = ( x_1  \, x_2 )$ 
     * $x_1$ : On-Base Percentage (OBP) , $x_2$ : Slugging Percentage (SLG)
* モデルパラメーター
  * b : バイアス項
  * ベクトル ${\bf w}^T = (w_1 \, w_2) $ 

### データとモデルを対応させるために

#### 行列、ベクトルによる表現

データセットのなかの複数のレコードをまとめて表現するために、従属変数をベクトル、独立変数を行列であらわすとプログラミングしやすくなる。

$$ {\bf y} = b + X {\bf w} $$

$$ {\bf y^{data}} = {\bf y} + {\bf \epsilon} $$

* 独立変数  
  * $X$ 行列 : i番目の行$(x_{i1} \, x_{i2})$が、i番目のレコードのなかのOLG, SLGに対応する。

In [4]:
x = tf.placeholder( tf.float32, [None, 2])

* 従属変数
  * $\bf y^{data}$ ベクトル : i番目の成分が、i番目のレコードのなかの得点に対応する。
  * $\bf y$ ベクトル : モデルによって計算された得点

In [5]:
yData = tf.placeholder( tf.float32, [ None, 1])
b = tf.Variable(tf.zeros([1]))
w = tf.Variable(tf.zeros([2,1]))
y = tf.add( b,  tf.matmul( x, w) )

#### 確率モデルとしての得点モデル

* データのなかの得点$\bf y^{data}$には、モデルでは表しきれない誤差（ばらつき、ゆらぎ、擾乱）$\bf \epsilon$ が含まれていると考える。

* 独立変数$x_1, x_2$に対して、どれだけの誤差$\epsilon$が出てくるかを決めることはできず、何らかの確率分布にしたがって$\epsilon$が分布していると考える。

## モデルパラメーターの推定法 -- 最尤法

最尤原理
* 「現実のデータは、確率モデルのなかで最大のものが実現された結果である」と仮定する。

最尤法
* データが得られる確率を尤度（もっともらしさ）$L$と呼び、その関数の引数がモデルパラメータであると考える。
* すると、尤度関数$L({\bf w})$を最大にするようなモデルパラメーター$\bf w$の値が現実で実現されている、ということになる。
* すなわち、$ \partial L / \partial {\bf w} = 0$ を解くことで、モデルパラメーター$\bf w$の値を決定できる。
* $L$は非常に小さくなるので、数値計算で解くときには、$L$の代わりに対数尤度$ log{L(\bf w)}$ を用いる。

誤差$\epsilon$が正規分布に従うと仮定すると、対数尤度は次のようになる。

$$log{L(\bf w)} \propto  | {\bf y - y^{data}} |^2 $$


### 誤差関数

機械学習の分野では、モデルパラメーターを決めるために最小化する関数を、「誤差関数 (error function)」と呼ぶ。

* 上記の得点のように、モデルを線形関数としてあらわすと、最尤法での誤差関数と、最小二乗法での誤差関数が一致する。
$$ E =  | {\bf y - y^{data}} |^2 $$ 

In [6]:
errFunc = tf.reduce_sum( tf.square( y - yData )  )


* しかし、ロジスティック回帰（のような他の機械学習）やディープラーニングは一致しない。

* そして、ロジスティック回帰やディープラーニングでは、最小二乗法ではなく、最尤法を用いてモデルパラメーターを決めている。

なので、ディープラーニングを含む機械学習分野では、最尤法などの統計的推論の基礎を理解しておくことで、見通しが良くなるのだと思う。

#### 計算機科学分野の用語は独特

ちなみに、誤差関数の代わりに損失関数(loss function)、目的関数(objective function)といった用語が使われることもある。

* また、特殊関数の誤差関数 $erf(x}$　と機械学習の誤差関数は、まったく別のものである。

### 誤差関数を最小化する方法

#### 勾配降下法

$${\bf w}^{new} \leftarrow {\bf w}^{old} - \eta \left( \frac{\partial E}{\partial {\bf w}} \right)_{\bf w^{old}}  $$ 

* $\partial E / \partial {\bf w}$を求めるときに、学習データのなかのレコードをひとつづつ使うアルゴリズムを「確率的勾配降下法」と呼ぶ。
* 学習データを複数のバッチに分割して、$\partial E / \partial {\bf w}$を求めるときに、ひとつづつバッチを使っていくアルゴリズムを「ミニバッチによる勾配降下法」と呼ぶ。

ここでは、ミニバッチによる勾配降下法で粗く最適化してから、勾配降下法で細かく最適化してみる。最適化アルゴリズムは、学習率$\eta$を自動調整するAdmaOptimizerを使う。

In [7]:
trainStep = tf.train.AdamOptimizer().minimize( errFunc)

In [8]:
class CyclicMinibatch :
    def __init__( self, data) :
        self.counter = 0
        self.data = data
    
    def nextBatch( self, sz) :
        if  self.counter + sz < len(self.data) :
            prevCounter = self.counter
            self.counter += sz
            return self.data[ prevCounter: self.counter]
        else :
            prevCounter = self.counter
            self.counter = 0
            return self.data[ prevCounter : len(self.data)]

testMinibatch = CyclicMinibatch( range(33))
print testMinibatch.nextBatch(5)
print testMinibatch.nextBatch(15)
print testMinibatch.nextBatch(20)
print testMinibatch.nextBatch(7)

[0, 1, 2, 3, 4]
[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
[0, 1, 2, 3, 4, 5, 6]


## 機械による学習

In [9]:
sess = tf.Session()
sess.run( tf.global_variables_initializer())

### 学習のためのデータ

In [10]:
trainX = np.array( zip( moneyball.OBP, moneyball.SLG))
numTrain, dummy =  trainX.shape

trainY = np.array( moneyball.RS)
trainY = trainY.reshape( [trainY.size, 1])

cyclicTrainY = CyclicMinibatch( trainY)
cyclicTrainX = CyclicMinibatch( trainX)

print trainY.shape, cyclicTrainY.nextBatch(10).shape
print trainX.shape, cyclicTrainX.nextBatch(10).shape

print trainY.mean()
print b

(902, 1) (10, 1)
(902, 2) (10, 2)
703.809312639
<tf.Variable 'Variable:0' shape=(1,) dtype=float32_ref>


### ミニバッチによる勾配降下法で粗く最適化

In [11]:
iterNum = 1000 * 100;
monitorInterval = iterNum / 20
batchSz = numTrain / 10

biasSet = tf.assign(b, tf.constant( [ trainY.mean()]))
sess.run( biasSet)

for i in range(1, iterNum+1) :
    batchX = cyclicTrainX.nextBatch( batchSz)
    batchY = cyclicTrainY.nextBatch( batchSz)
    sess.run( trainStep, feed_dict={ x:batchX, yData:batchY})
    if i % monitorInterval == 0:
        errFuncVal = sess.run( errFunc, feed_dict={ x:trainX, yData:trainY})
        rmse = sqrt( errFuncVal / numTrain  )
        print 'Step : %d, RMSE : %f' % ( i, rmse)

Step : 5000, RMSE : 93.239365
Step : 10000, RMSE : 93.218053
Step : 15000, RMSE : 93.198448
Step : 20000, RMSE : 93.180412
Step : 25000, RMSE : 93.162787
Step : 30000, RMSE : 93.145590
Step : 35000, RMSE : 93.128845
Step : 40000, RMSE : 93.112225
Step : 45000, RMSE : 93.095968
Step : 50000, RMSE : 93.080056
Step : 55000, RMSE : 93.064119
Step : 60000, RMSE : 93.048097
Step : 65000, RMSE : 93.032241
Step : 70000, RMSE : 93.016512
Step : 75000, RMSE : 93.000939
Step : 80000, RMSE : 92.985464
Step : 85000, RMSE : 92.970103
Step : 90000, RMSE : 92.954868
Step : 95000, RMSE : 92.939711
Step : 100000, RMSE : 92.924638


### 勾配降下法で最適化

In [12]:
rmseStart = rmse
prevRmse = rmse * 10.0
monitorInterval = 200000
i = 0

while( abs(prevRmse - rmse) > rmse * 1.0e-8 ) :
    i += 1
    sess.run( trainStep, feed_dict={ x:trainX, yData:trainY})
    if i % monitorInterval == 0:
        errFuncVal = sess.run( errFunc, feed_dict={ x:trainX, yData:trainY})
        prevRmse = rmse
        rmse = sqrt( errFuncVal / numTrain  )
        print 'Step : %d, RMSE : %f' % ( i, rmse)

Step : 200000, RMSE : 84.037484
Step : 400000, RMSE : 75.230149
Step : 600000, RMSE : 66.660061
Step : 800000, RMSE : 58.414624
Step : 1000000, RMSE : 50.449839
Step : 1200000, RMSE : 42.922224
Step : 1400000, RMSE : 36.106260
Step : 1600000, RMSE : 30.483099
Step : 1800000, RMSE : 26.813997
Step : 2000000, RMSE : 25.714320
Step : 2200000, RMSE : 25.289685
Step : 2400000, RMSE : 24.984604
Step : 2600000, RMSE : 24.803507
Step : 2800000, RMSE : 24.748757
Step : 3000000, RMSE : 24.748757


## モデル式のテスト

#### 今回、線形回帰で求めた式

In [13]:
estY = sess.run(b)
estW = sess.run(w)
print "Model formula : y = %f + %f x1 + %f x2" % (estY[0] , estW[0], estW[1])

Model formula : y = -804.627075 + 2737.767822 x1 + 1584.908569 x2


#### [edX MITx:15071x The Analytics Edge](https://courses.edx.org/courses/course-v1:MITx+15.071x_3+1T2016/courseware/f8d71d64418146f18a066d7f0379678c/6324edb8a22c4e35b937490647bfe203/?activate_block_id=block-v1%3AMITx%2B15.071x_3%2B1T2016%2Btype%40sequential%2Bblock%406324edb8a22c4e35b937490647bfe203)の演習で、Rを使って求めた式

$$ y =  -804.63 + 2737.77 x_1 + 1584.91 x_2 $$


両者は、ほぼ一致している。

（正式のデータ分析であれば、テスト用のデータを2002年の試合データから取り出して、モデル式の精度をテストするところだが、今回はこれでよしとする。）

# まとめ

TensorFlow の基本を学び、正しく動作するpythonスクリプトを書くことができた。